# Data access and cleaning
Will access data files and apis, clean data, and present data to be analized

In [2]:
# Import Dependencies
import pandas as pd



,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [4]:
# Load data files

# National Park Data: Parks names and locations
nps_parks_csv = "../data/nps/parks.csv"
nps_parks_df = pd.read_csv(nps_parks_csv)

# National Park Data: Parks Species 
# WARNING WILL THROW A WARNING BECAUSE OF EXTRA COMMA AT EOL
nps_species_csv = "../data/nps/species.csv"
nps_species_df = pd.read_csv(nps_species_csv)

# species file has an extra comma at the end of each line so need to remove that by the index
column_numbers = [x for x in range(nps_species_df.shape[1])]
column_numbers.remove(13) #removing column integer index 13
nps_species_df = nps_species_df.iloc[:, column_numbers] #return all columns except the 13th column

nps_species_df.head()

C:\Users\bgald\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN
